In [36]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [37]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res.json")):
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        with open(os.path.join(sub_folder_path, "result.json")) as f:
            last_text = f.readlines()[-1]
            epochs_trained = re.search('epoch": [0-9]*', last_text).group(0).split(": ")[-1]
        params["folder"] = folder
        params["epochs_trained"] = int(epochs_trained)
        trials.append((params, eval_res))
open_trials = [folder for folder in os.listdir(ray_folder) if folder.startswith("hyperopt")]
trials_run_folders = [f[0]["folder"] for f in trials]
open_trials = [f for f in open_trials if f not in trials_run_folders]
print(f"Trials run: {len(trials)} Trials still open: {len(open_trials)}")
with open("still_open_qm9_full_full_mat.txt", "w") as f: 
    f.write("\n".join(open_trials))

Trials run: 68 Trials still open: 32


In [38]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials: 
    print(trial[0]["folder"])
    print(trial[1]["cummulative_stats"])
    print(trial[0]["epochs_trained"])
    print("---")


Found 68 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_full_testing/tune_logs/MGNN_hyp_small_full_full_mat_loss.py
hyperopt_train_036e6_00067_67_batch_size=8,data_aug_factor=4,edge_threshold_val=2.9581,hidden_dim=128,lr=0.0006,message_net_dropou_2025-07-18_15-04-17
{'energy_abs': {'mean': -1083.583808018691, 'std': 274.82301925241757}, 'iterations': {'mean': 11.98, 'std': 2.0831706603156643}, 'energy_rel': {'mean': -0.6895726225343559, 'std': 0.03935376863210684}, 'diis': {'mean': 0.20393201893621662, 'std': 0.031054901739961612}, 'rmse': {'mean': 0.008490712640139201, 'std': 0.0025700245868826395}}
5
---
hyperopt_train_036e6_00078_78_batch_size=8,data_aug_factor=1,edge_threshold_val=2.5786,hidden_dim=256,lr=0.0009,message_net_dropou_2025-07-18_15-39-03
{'energy_abs': {'mean': -1058.4261940079916, 'std': 264.1905437171964}, 'iterations': {'mean': 12.04, 'std': 1.9693653800145872}, 'energy_rel': {'mean': -0.6748333923310406, 'std': 0.03502659846108127}, 'dii

In [39]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.3f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 12.0(2.1) & -1083.6(274.8) & -0.690(0.039) & 0.204(0.031) & 0.0085(0.0026) \\
1 & 12.0(2.0) & -1058.4(264.2) & -0.675(0.035) & 0.196(0.030) & 0.0090(0.0024) \\
2 & 12.1(1.8) & -1081.6(274.1) & -0.688(0.040) & 0.204(0.028) & 0.0091(0.0027) \\
3 & 12.1(2.2) & -1076.6(274.5) & -0.685(0.040) & 0.200(0.026) & 0.0087(0.0027) \\
4 & 12.2(1.7) & -1086.2(271.7) & -0.692(0.036) & 0.207(0.030) & 0.0094(0.0028) \\
5 & 12.2(2.4) & -1057.7(265.8) & -0.674(0.036) & 0.197(0.028) & 0.0094(0.0029) \\
6 & 12.2(2.1) & -1068.8(269.3) & -0.681(0.040) & 0.202(0.032) & 0.0089(0.0029) \\
7 & 12.3(2.2) & -1068.2(273.0) & -0.680(0.042) & 0.202(0.027) & 0.0098(0.0033) \\
8 & 12.3(2.0) & -1079.4(271.8) & -0.687(0.037) & 0.204(0.029) & 0.0095(0.0029) \\
9 & 12.3(2.3) & -1078.4(275.0) & -0.686(0.041) & 0.205(0.027) & 0.0093(0.0032) \\
\bottomrule
\end{tabular}



In [40]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 128 & 256 & 512 & 128 & 128 & 256 & 128 & 512 & 256 & 512 \\
Batch Size & 8 & 8 & 16 & 8 & 16 & 8 & 8 & 32 & 32 & 16 \\
Data aug. & 4.00 & 1.00 & 3.00 & 2.00 & 2.00 & 1.00 & 4.00 & 4.00 & 3.00 & 3.00 \\
Edge thres. & 2.96 & 2.58 & 3.54 & 2.85 & 3.57 & 3.21 & 3.27 & 3.24 & 2.67 & 2.72 \\
Message p. steps & 5 & 5 & 4 & 2 & 3 & 2 & 3 & 2 & 2 & 3 \\
Message Net Dropout & 0.08 & 0.16 & 0.23 & 0.11 & 0.27 & 0.21 & 0.16 & 0.12 & 0.20 & 0.04 \\
Message Net Layers & 3 & 3 & 2 & 4 & 3 & 3 & 2 & 4 & 2 & 2 \\
Learn Rate & 5.62e-04 & 9.31e-04 & 3.84e-04 & 8.03e-04 & 9.04e-04 & 2.75e-04 & 6.98e-04 & 3.99e-04 & 2.42e-03 & 5.85e-04 \\
Weight Decay & 5.61e-05 & 5.46e-05 & 9.63e-05 & 6.06e-04 & 2.13e-05 & 4.69e-05 & 8.74e-04 & 2.38e-05 & 4.19e-05 & 1.45e-04 \\
\bottomrule
\end{tabular}

